# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import time
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.amax(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.09000000171363354


## 5. DDPG

**Credit:** I have the DDPG code from DDPG_Pendulum folder, and made some appropriate changes to tweak the hyperparemeters

In [6]:
from ddpg_agent_updated import Agent
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
    
# self, num_agents,state_size, action_size, random_seed, gamma=GAMMA, tau= TAU, lr_actor=LR_ACTOR, lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY, mu=0., theta=0.15, sigma=0.2, learn_rate=LEARNING_RATE, time_update = TIME_UPDATE, eps_max=EPS_MAX, eps_min=EPS_MIN, eps_decay=EPS_DECAY
# agents = Agent(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=1, gamma=0.99, tau=1e-3, lr_actor=1e-4, lr_critic=1e-3, weight_decay=0., mu=0., theta=0.12, sigma=0.2, learn_rate=10, time_update=10, eps_max=1.0, eps_min=0.1, eps_decay=0.)
# agents = Agent(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=1, gamma=0.99, tau=8e-3, lr_actor=1e-4, lr_critic=1e-3, weight_decay=0., mu=0., theta=0.12, sigma=0.2, learn_rate=10, time_update=10, eps_max=1.0, eps_min=0.1, eps_decay=0.)
agents = Agent(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=1, gamma=0.99, tau=8e-2, lr_actor=2e-4, lr_critic=2e-3, weight_decay=0., mu=0., theta=0.12, sigma=0.2, learn_rate=10, time_update=10, eps_max=1.0, eps_min=0.1, eps_decay=0.)

In [ ]:
# This cell was executed and the results of the cell are below. However for reasons unknown to me the cell is not showing a number or *.
            
def ddpg(n_episodes=2000, max_t=10000, print_every=100):
    ''' DDPG Algorithm
        
        Params
            n_episode (int): maximum number of training episodes
            max_t (int): maximum number of timesteps per episode
            print_every (int) : frequency of printing information throughout iteration 
    '''
    
    individual_max_scores = []                                      # Max scores from each episode
    mean_scores = []
    scores_window = deque(maxlen=print_every)
    times_window = deque(maxlen=print_every)
    
    i_episode = 0
        
#     for i_episode in range(1, n_episodes+1):
    while True:
        
        i_episode += 1
        
        env_info = env.reset(train_mode=True)[brain_name]     # Reset the environment    
        states = env_info.vector_observations                 # Get the current states (for each agent)        
        
        agents.reset()
        
        scores = np.zeros(num_agents)                       # Reset the score
        start_time = time.time()                              # Record start time
        
        t = 0
        while True:
#         for t in range(max_t):
            actions = agents.act(states) # Action for the state
            env_info = env.step(actions)[brain_name]     # Get the new details of the environment
            
            next_states = env_info.vector_observations      # Get the next state
            rewards = env_info.rewards                   # Get the reward
            dones = env_info.local_done                    # Get the done value
            
            agents.step(t, states, actions, rewards, next_states, dones)
                
            states = next_states
            scores += rewards
            
            t += 1
            
            # Check if the episode is done
            if np.any(dones):
                break 
        
        # calculate the mean score of all the agents for this episode and add to the mean
        individual_max_scores.append(np.amax(scores)) 
        
        # add the max score of all the agents for this episode to the sliding window
        scores_window.append(np.amax(scores))
        
        # add the time for this episode to the sliding window
        times_window.append(time.time()-start_time)
        
        # The mean of the scores window
        last_mean = np.mean(scores_window)
        
        mean_scores.append(last_mean)
        
        # The mean of the times window
        mean_time = np.mean(times_window)
        
        print('\rEpisode {}\tSliding Max: {:.2f}\tNon-zero Count: {:d}\tAverage Score: {:.4f}'.format(i_episode,np.amax(scores_window), np.count_nonzero(scores_window),last_mean), end="")
        agents.save()
        
        if i_episode % print_every == 0:
            print('\rEpisode {}\tSliding Max: {:.2f}\tNon-zero Count: {:d}\tAverage Score: {:.4f}'.format(i_episode,np.amax(scores_window),np.count_nonzero(scores_window),last_mean))
        
        if last_mean > 0.5 and i_episode > 100:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.3f}'.format(i_episode, last_mean))
            agents.save()
            break
            
    return individual_max_scores, mean_scores
#1000000
#100000
#10000
#50000
scores, mean_scores = ddpg(n_episodes=50000, max_t=10000000, print_every=100)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores,'b',label='Episode Scores')
plt.plot(np.arange(1, len(scores)+1), mean_scores,'r',\
         linewidth=1,label='Avg. score')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Sliding Max: 0.09	Non-zero Count: 2	Average Score: 0.0018
Episode 200	Sliding Max: 0.20	Non-zero Count: 22	Average Score: 0.0234
Episode 300	Sliding Max: 0.20	Non-zero Count: 10	Average Score: 0.0127
Episode 400	Sliding Max: 0.20	Non-zero Count: 13	Average Score: 0.0137
Episode 500	Sliding Max: 0.10	Non-zero Count: 46	Average Score: 0.0438
Episode 600	Sliding Max: 0.10	Non-zero Count: 19	Average Score: 0.0188
Episode 700	Sliding Max: 0.10	Non-zero Count: 37	Average Score: 0.0355
Episode 794	Sliding Max: 0.10	Non-zero Count: 42	Average Score: 0.0402

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

When finished, you can close the environment.

In [ ]:
env.close()